In [29]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from datetime import datetime
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import precision_score, recall_score
from tensorflow.keras.models import load_model
from glob import glob

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [30]:
train_path = 'Original\Train'
test_path = 'Original\Test'

In [31]:
vgg = VGG16(input_shape=[224, 224, 3], weights='imagenet', include_top=False)

In [32]:
for layer in vgg.layers:
    layer.trainable = False

In [33]:
folders = glob('Original\Train\*')
print(len(folders))

2


In [34]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='sigmoid')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [35]:
adam = optimizers.Adam()
model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [36]:
data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input
)

In [37]:
training_data = data_generator.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 8,
                                                 class_mode = 'categorical')

Found 4473 images belonging to 2 classes.


In [38]:
testing_data = data_generator.flow_from_directory(test_path,
                                                 target_size = (224, 224),
                                                 batch_size = 8,
                                                 class_mode = 'categorical')

Found 651 images belonging to 2 classes.


In [39]:
checkpoint = ModelCheckpoint(filepath = 'Model1.h5', verbose = 2, save_best_only = True)

In [40]:
callbacks = [checkpoint]

start = datetime.now()

model_history = model.fit(
    training_data,
    validation_data = testing_data,
    epochs = 10,
    steps_per_epoch = 5,
    validation_steps = 32,
    callbacks = callbacks, verbose = 1,
)

duration = datetime.now() - start

print('Training time: ' , duration)

Epoch 1/10
5/5 [==============================] - ETA: 0s - loss: 1.5807 - accuracy: 0.8500
Epoch 1: val_loss improved from inf to 2.65325, saving model to Model1.h5
5/5 [==============================] - 26s 6s/step - loss: 1.5807 - accuracy: 0.8500 - val_loss: 2.6533 - val_accuracy: 0.8828
Epoch 2/10


C:\Users\migue\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5/5 [==============================] - ETA: 0s - loss: 3.1302 - accuracy: 0.8250
Epoch 2: val_loss did not improve from 2.65325
5/5 [==============================] - 25s 6s/step - loss: 3.1302 - accuracy: 0.8250 - val_loss: 3.7060 - val_accuracy: 0.5508
Epoch 3/10
5/5 [==============================] - ETA: 0s - loss: 1.1584 - accuracy: 0.8750
Epoch 3: val_loss improved from 2.65325 to 1.66883, saving model to Model1.h5
5/5 [==============================] - 25s 6s/step - loss: 1.1584 - accuracy: 0.8750 - val_loss: 1.6688 - val_accuracy: 0.7734
Epoch 4/10
5/5 [==============================] - ETA: 0s - loss: 1.0928 - accuracy: 0.8750
Epoch 4: val_loss did not improve from 1.66883
5/5 [==============================] - 25s 6s/step - loss: 1.0928 - accuracy: 0.8750 - val_loss: 2.3382 - val_accuracy: 0.8398
Epoch 5/10
5/5 [==============================] - ETA: 0s - loss: 1.1447 - accuracy: 0.8750
Epoch 5: val_loss improved from 1.66883 to 1.62975, saving model to Model1.h5
5/5 [=======

//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

In [41]:
model = load_model('model1.h5')

In [ ]:
predictions = model.predict(testing_data)

predicted_classes = predictions.argmax(axis=1)

true_classes = testing_data.classes

precision = precision_score(true_classes, predicted_classes)
recall = recall_score(true_classes, predicted_classes)

print("Precision:", precision)
print("Recall:", recall)

//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

                                                                                                                      ARTIFICIALLY INCREASE THE BENIGN SAMPLE SIZE

In [21]:
# from PIL import Image
# import os
# import random

# benign_dir = 'Original/Train/Benign'

# def rotate_and_save(image_path, output_path, rotation_range=(-40, 40)):
#     # Open the image
#     image = Image.open(image_path)
    
#     # Randomly choose rotation angle within the specified range
#     rotation_angle = random.randint(rotation_range[0], rotation_range[1])
    
#     # Rotate the image
#     rotated_image = image.rotate(rotation_angle)
    
#     # Save the rotated image
#     rotated_image.save(output_path)

# # Loop through each benign image and apply rotation
# for filename in os.listdir(benign_dir):
#     if filename.endswith('.jpg') or filename.endswith('.png'):  # Adjust file extensions as needed
#         # Path to the original image
#         image_path = os.path.join(benign_dir, filename)
        
#         # Path for the rotated image (you can adjust how you want to name the rotated files)
#         output_path = os.path.join(benign_dir, 'rotated_3' + filename)
        
#         # Apply rotation and save the rotated image
#         rotate_and_save(image_path, output_path)
